In [1]:
import pandas as pd
import re

def convert_excel_to_html(excel_path, output_html="schedule_tokyo_trip.html"):
    df = pd.read_excel(excel_path)

    region_colors = {
        "긴자": ("#E0ECFF", "#1E3A8A"),
        "도쿄역": ("#E8F5E9", "#2E7D32"),
        "마루노우치": ("#FFF8E1", "#EF6C00"),
        "아자부다이": ("#F3E5F5", "#6A1B9A"),
        "시부야": ("#E1F5FE", "#01579B"),
        "아키하바라": ("#FFF3E0", "#E65100"),
        "교바시": ("#E8EAF6", "#283593"),
    }

    days = df["day"].dropna().unique()
    html = """
    <!DOCTYPE html>
    <html lang="ko">
    <head>
    <meta charset="utf-8">
    <title>Tokyo MVP Global Experience Schedule</title>
    <style>
        body {
            font-family: 'Pretendard', 'Apple SD Gothic Neo', Arial, sans-serif;
            margin: 0;
            background: #fff5f5;
            color: #222;
            line-height: 1.65;
        }
        header {
            text-align: center;
            padding: 28px 0 16px;
            background: linear-gradient(90deg, #e57373, #ef9a9a);
            color: #fff;
        }
        h1 { margin: 0; font-size: 1.8em; }
        .tab-bar {
            text-align: center;
            background: #ffecec;
            padding: 10px 0;
            position: sticky;
            top: 0;
            z-index: 10;
            box-shadow: 0 2px 4px rgba(0,0,0,0.05);
        }
        .tab-button {
            background: #fff;
            border: 1px solid #ffcdd2;
            border-radius: 20px;
            margin: 4px 6px;
            padding: 8px 18px;
            font-size: 0.95em;
            cursor: pointer;
            transition: all 0.2s ease;
            color: #d32f2f;
        }
        .tab-button:hover { background: #ffe5e5; }
        .tab-button.active {
            background: #ef5350;
            color: #fff;
            border-color: #e57373;
            font-weight: 600;
        }
        .day-section {
            display: none;
            max-width: 1080px;
            margin: 0 auto;
            padding: 25px;
        }
        .day-section.active { display: block; }
        .day-header {
            font-size: 1.4em;
            color: #c62828;
            border-left: 6px solid #ef9a9a;
            background: #ffecec;
            padding: 8px 14px;
            border-radius: 6px;
            margin: 20px 0 25px;
            font-weight: 600;
        }

        .schedule-item {
            background: #fff;
            border-radius: 12px;
            padding: 22px 28px 26px 28px;
            margin: 22px 0;
            box-shadow: 0 3px 6px rgba(0,0,0,0.08);
            position: relative;
            transition: all 0.2s ease;
        }
        .schedule-item:hover {
            transform: translateY(-3px);
            box-shadow: 0 6px 12px rgba(0,0,0,0.12);
        }
        .region-label {
            position: absolute;
            top: 12px;
            right: 26px;
            font-weight: 700;
            font-size: 0.85em;
            padding: 4px 10px;
            border-radius: 6px;
        }
        .location-line {
            font-weight: 600;
            font-size: 1.05em;
            color: #1a1a1a;
            margin-bottom: 10px;
        }
        .location a {
            color: #d32f2f;
            text-decoration: none;
            font-size: 1.5em;
            font-weight: 800;
        }
        .location a:hover { text-decoration: underline; }

        .content-box {
            background: #fff8f8;
            border-left: 5px solid #ef9a9a;
            border-radius: 6px;
            margin: 10px 0;
            padding: 10px 14px;
        }
        .content-title {
            display: inline-block;
            background: #ef9a9a;
            color: #fff;
            font-weight: 700;
            border-radius: 6px;
            padding: 2px 10px;
            margin-bottom: 6px;
        }
        .content-text { margin: 4px 0 0 4px; font-size: 0.95em; }

        .team-quest-box {
            background: #ffecec;
            border-left: 5px solid #e57373;
            border-radius: 6px;
            margin: 14px 0;
            padding: 12px 16px;
        }
        .team-quest-title {
            display: inline-block;
            background: #e57373;
            color: #fff;
            font-weight: 700;
            border-radius: 6px;
            padding: 2px 10px;
            margin-bottom: 6px;
        }
        .team-quest-box .content-text {
            font-weight: 600;
            font-size: 1.0em;
            color: #333;
            line-height: 1.65;
            margin-top: 4px;
        }
        /* CAPTURE */
        .capture-box {
            background: #EAF4FE;
            border-left: 5px solid #64B5F6;   /* ✅ Team Quest와 동일 두께로 맞춤 */
            border-radius: 6px;
            margin: 14px 0;
            padding: 12px 16px;
            margin-left: -5px;                /* ✅ border 두께만큼 왼쪽으로 당김 */
        }
        .capture-title {
            display: inline-block;
            background: #64B5F6;
            color: #fff;
            font-weight: 700;
            border-radius: 6px;
            padding: 2px 10px;
            margin-bottom: 6px;
        }
        .capture-box ul {
            margin: 6px 0 0 18px;
            list-style-type: disc;
            font-style: italic;
        }
        .capture-box li {
            margin-bottom: 4px;
            line-height: 1.6;
        }
        
        /* TALK & THINK */
        .think-box {
            background: #E8F5E9;
            border-left: 5px solid #81C784;   /* ✅ 동일 두께 */
            border-radius: 6px;
            margin: 14px 0;
            padding: 12px 16px;
            margin-left: -5px;                /* ✅ Team Quest와 정확히 정렬 */
        }
        .think-title {
            display: inline-block;
            background: #81C784;
            color: #fff;
            font-weight: 700;
            border-radius: 6px;
            padding: 2px 10px;
            margin-bottom: 6px;
        }
        .think-box ul {
            margin: 6px 0 0 18px;
            list-style-type: disc;
            font-style: italic;
        }
        .think-box li {
            margin-bottom: 4px;
            line-height: 1.6;
        }
    </style>

    <script>
        function showDay(dayId) {
            const sections = document.querySelectorAll('.day-section');
            const buttons = document.querySelectorAll('.tab-button');
            sections.forEach(sec => sec.classList.remove('active'));
            buttons.forEach(btn => btn.classList.remove('active'));
            document.getElementById(dayId).classList.add('active');
            document.getElementById('btn-' + dayId).classList.add('active');
        }
        window.onload = () => {
            const first = document.querySelector('.day-section');
            if (first) showDay(first.id);
        };
    </script>
    </head>
    <body>
    <header>
        <h1>🌏 LG MVP Global Customer Experience (TEAM 3)</h1>
    </header>
    <div class="tab-bar">
    """

    for day in days:
        html += f"<button id='btn-{day}' class='tab-button' onclick=\"showDay('{day}')\">{day}</button>"
    html += "</div>"

    for day in days:
        html += f"<div class='day-section' id='{day}'><div class='day-header'>{day}</div>"
        sub = df[df["day"] == day]

        for _, row in sub.iterrows():
            region = row.get('region')
            desc = f"{row['description']}" if pd.notna(row.get('description')) else ""
            quest_num = str(row['quest_num']).strip() if pd.notna(row.get('quest_num')) else None

            region_html = ""
            if pd.notna(region) and region in region_colors:
                bg, fg = region_colors[region]
                region_html = f"<div class='region-label' style='background:{bg};color:{fg};'>{region}</div>"

            location_html = row["location"]
            if pd.notna(row.get("link_map")):
                location_html = f"<a href='{row['link_map']}' target='_blank'>{row['location']}</a>"

            html += f"<div class='schedule-item'>{region_html}<div class='location-line'><span class='location'>{location_html}</span>&nbsp;&nbsp;&nbsp;{desc}</div>"

            # 기본 콘텐츠
            content_fields = [
                ("고객 경험", "customer_experience"),
                ("공간 및 서비스 특징", "space_features"),
                ("주요 관점", "key_perspective"),
                ("필드 가이드", "field_guide"),
            ]
            for label, field in content_fields:
                value = row.get(field)
                if pd.notna(value):
                    html += f"<div class='content-box'><div class='content-title'>{label}</div><div class='content-text'>{value}</div></div>"

            # Team Quest
            if pd.notna(row.get("team_quest")):
                num_html = f"<span class='num'>#{quest_num}</span>" if quest_num else ""
                html += f"<div class='team-quest-box'><div class='team-quest-title'>Team Quest {num_html}</div><div class='content-text'>{row['team_quest']}</div></div>"

            # CAPTURE
            if pd.notna(row.get("cco")):
                cco_text = str(row["cco"]).strip()
                items = [i.strip('- ').strip() for i in cco_text.split('\n') if i.strip()]
                items_html = "".join([f"<li>{i}</li>" for i in items])
                html += f"<div class='capture-box'><div class='capture-title'>체험 공간과 순간 (Capture)</div><ul>{items_html}</ul></div>"

            # TALK & THINK
            if pd.notna(row.get("cto")):
                cto_text = str(row["cto"]).strip()
                items = [i.strip('- ').strip() for i in cto_text.split('\n') if i.strip()]
                items_html = "".join([f"<li>{i}</li>" for i in items])
                html += f"<div class='think-box'><div class='think-title'>체험을 통한 발견 (TALK & THINK)</div><ul>{items_html}</ul></div>"

            html += "</div>"  # schedule-item end

        html += "</div>"  # day-section end

    html += "</body></html>"

    with open(output_html, "w", encoding="utf-8") as f:
        f.write(html)

    return output_html


# 실행 예시
html_path = convert_excel_to_html("schedule_tokyo_trip.xlsx")
print("✅ HTML 생성 완료:", html_path)

✅ HTML 생성 완료: schedule_tokyo_trip.html
